Generate data untuk mengecek waktu eksekusi berdasarkan jumlah node

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import random
from itertools import permutations

# Definisi Kelas Komunitas

In [147]:
# number_of_node merupakan jumlah node pada komunitas, pada nilai tersebut nilai minimalnya adalah 2
# comm_id merupakan ID yang digunakan ketika membuat objek, default valuenya adalah A

# prob_connection merupakan persentasi jumlah relasi pada setiap node dan berbentuk array
# sebagai contoh misalkan terdapat 3 node, maka prob_connection nya dapat diisi menjadi [0.1, 0.2, 0.3]
# hal tersebut memiliki arti node 1 memiliki jumlah relasi 0.1 * total relasi yang memungkinkan
# pada contoh ini relasi yang memungkinkan pada setiap node adalah 2 (relasi ke diri sendiri tidak termasuk)
# default value dari prob_connection adalah 0, jadi jika prob_connection tidak diisi dengan array,
# maka persentase relasi tiap node akan dirandom

class Community:
  def __init__ (self,number_of_node, comm_id='A', prob_connection=0):
    self.comm_id = comm_id

    if number_of_node > 1:
      self.number_of_node = number_of_node
    else :
      raise ValueError('Jumlah node minimal adalah 2')

    self.prob_connection = prob_connection
    self.matrix_relation = 0
    self.table_of_node = Community.generate_table_node(self, self.number_of_node)
    self.table_of_relation = Community.generate_table_relation(self)

  # fungsi untuk generate tabel yang berisi daftar node pada suatu komunitas
  def generate_table_node(self, number_node):
    comm_id = self.comm_id
    df = pd.DataFrame(columns=['Comm_ID','Node_ID','X','Y','Trust','RAM'])
    for i in range(number_node):
      df = df.append({'Comm_ID':comm_id, 'Node_ID':comm_id+'_' + str(i), 'X':random.randint(0,100),'Y':random.randint(0,100), 'Trust':round(random.uniform(0.5,1.0),3), 'RAM':random.uniform(0.1, 4.0)}, ignore_index = True)

    ran_comp_res = Community.generate_comp_resource(self)

    for i in ran_comp_res:
      val_class = ran_comp_res[i]
      for j in val_class:
        df.at[j, 'Comp_Res'] = i

    return df

  # fungsi untuk generate computation resource
  def generate_comp_resource(self):
    val = self.number_of_node
    if (val > 3):
      ls_rand_gen = [0.1, 0.15, 0.35, 0.3]
      ls_ha = []

      if (val > 4):
        for i in ls_rand_gen:
          ls_ha.append(int(np.ceil(i * val)))

        if (sum(ls_ha) > val):
          ls_ha[2] -= (sum(ls_ha) - val)
        elif (sum(ls_ha) < val):
          ls_ha[2] += (val - sum(ls_ha))

        if (ls_ha[3] > ls_ha[2]):
          ls_ha[2], ls_ha[3] = ls_ha[3], ls_ha[2]
      elif (val == 4):
        ls_ha = [1,1,1,1]

      dict_res = {}
      ls_index = range(val)
      for i,val in enumerate(ls_ha):
        res_random = random.sample(ls_index, val)
        dict_res['Class-'+str(i+1)]  = res_random
        ls_index = list(set(ls_index)-set(res_random))

      return dict_res

    elif (val == 3):
      dict_res = {
          'Class-1':[0],
          'Class-2':[1],
          'Class-3':[2]
      }
 
      return dict_res

    elif (val == 2):
      dict_res = {
          'Class-1':[0],
          'Class-2':[1],
      }

      return dict_res

  # fungsi untuk membuat matrik relasi node
  def generate_matrix_relation(self):
    number_node = self.number_of_node
    prob_connection = self.prob_connection
    mat_relation = np.identity(number_node) 
    all_con_posible = number_node -1

    if (prob_connection == 0):
      new_prob_con = []
      
      for i in range(number_node):
        new_prob_con.append(random.random())
      prob_connection = new_prob_con
    

    for i in range(number_node):
      list_row = list(mat_relation[i,:])

      list_pos_con = []
      list_already_con = []

      for n,val in enumerate(list_row):
        if val == 0:
          list_pos_con.append(n)
        elif val == 1 and n != i:
          list_already_con.append(n)

      number_con = int(np.ceil(prob_connection[i] * all_con_posible))
      random_con = 0

      if (len(list_already_con) < number_con):
        con_left = number_con - (all_con_posible - len(list_pos_con))
        rand_con = random.sample(list_pos_con, con_left)
        
        for j in rand_con:
          mat_relation[j,i] = 1
          mat_relation[i,j] = 1

    self.matrix_relation = mat_relation
    return mat_relation
  
  # menambahkan quality pada matrik connection
  def add_quality_relation(mat_relation):
    mat_copy = mat_relation.copy()
    for i in range(len(mat_relation)):
      for n,val in enumerate(list(mat_relation[i,:])):
        if (n > i and val == 1):
          rand = round(random.uniform(0.5,1.0),3)
          mat_copy[n,i] = rand
          mat_copy[i,n] = rand    
    return mat_copy 

  # fungsi untuk generate tabel yang berisi daftar koneksi tiap node
  def generate_table_relation(self):
    number_node = self.number_of_node
    mat_relation = Community.generate_matrix_relation(self)

    mat_quality_relation = Community.add_quality_relation(mat_relation)

    df = pd.DataFrame(columns=['node_1','node_2','quality','datetime'])

    for i in range(number_node):
      list_node_rel = list(mat_quality_relation[i,:])
      for n,val in enumerate(list_node_rel):
        if (n != i and val != 0):
          df = df.append({'node_1':'A_'+ str(i), 'node_2':'A_' + str(n), 'quality':val,'datetime':dt.datetime.now()}, ignore_index = True)

    return df

  #fungsi untuk kalkulasi
  def calculate_table(self, alpha=0.3, beta=0.3, gamma=0.4):
    df_calculate = self.table_of_relation.groupby(['node_1']).sum()
    df_calculate['relation'] = list(self.table_of_relation.groupby(['node_1']).size())
    df_calculate['C_score'] = df_calculate['quality']/df_calculate['relation']
    
    df_node = self.table_of_node.copy()
    df_node['comp_rate'] = df_node['RAM'].apply(Community.parse_ram)
    df_node['resource_rate'] = df_node['Comp_Res'].apply(Community.parse_comp_res)

    df_calculate['comp_rate'] = df_node['comp_rate'].tolist()
    df_calculate['resource_rate'] = df_node['resource_rate'].tolist()
    df_calculate['T_score'] = self.table_of_node['Trust'].tolist()
    df_calculate['I_score'] = (df_calculate['comp_rate']+df_calculate['resource_rate'])/2
    df_calculate['S_score'] = (alpha*df_calculate['C_score'])+(beta*df_calculate['I_score'])+(gamma*df_calculate['T_score'])

    return df_calculate.sort_values(by='S_score', ascending=False)

  def parse_ram(ram):
    pwr = 0
    if ram < 1 :
        pwr = 0.3
    if ram >= 1 :
        pwr = 0.6
    if ram >=4 :
        pwr = 1
    return pwr

  def parse_comp_res(class_type):
    val = 0
    if (class_type == 'Class-1'):
      val = 0.8
    elif (class_type == 'Class-2'):
      val = 0.6
    elif (class_type == 'Class-3'):
      val = 0.4
    elif (class_type == 'Class-4'):
      val = 0.2
    
    return val

## Buat objek komunitas

In [149]:
coba = Community(number_of_node=5, comm_id='B', prob_connection=[0.1, 0.3, 0.5, 0.4, 0.5])

In [150]:
coba.table_of_node

,Comm_ID,Node_ID,X,Y,Trust,RAM,Comp_Res
0,B,B_0,55,84,0.671,0.364740,Class-3
1,B,B_1,7,31,0.740,2.807242,Class-4
2,B,B_2,23,83,0.519,1.800675,Class-3
3,B,B_3,39,70,0.952,1.098204,Class-1
4,B,B_4,39,76,0.657,2.697810,Class-2


In [151]:
coba.matrix_relation

array([[1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 0.],
       [0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 0.],
       [1., 0., 1., 0., 1.]])

In [152]:
 coba.table_of_relation

,node_1,node_2,quality,datetime
0,A_0,A_1,0.815,2021-10-09 06:31:32.746587
1,A_0,A_3,0.827,2021-10-09 06:31:32.750096
2,A_0,A_4,0.925,2021-10-09 06:31:32.753487
3,A_1,A_0,0.815,2021-10-09 06:31:32.756793
4,A_1,A_2,0.702,2021-10-09 06:31:32.760050
5,A_2,A_1,0.702,2021-10-09 06:31:32.763338
6,A_2,A_3,0.512,2021-10-09 06:31:32.766804
7,A_2,A_4,0.550,2021-10-09 06:31:32.770124
8,A_3,A_0,0.827,2021-10-09 06:31:32.773539
9,A_3,A_2,0.512,2021-10-09 06:31:32.777087


In [153]:
coba.calculate_table()

,quality,relation,C_score,comp_rate,resource_rate,T_score,I_score,S_score
node_1,,,,,,,,
A_3,1.339,2,0.669500,0.6,0.8,0.952,0.70,0.79165
A_4,1.475,2,0.737500,0.6,0.6,0.657,0.60,0.66405
A_1,1.517,2,0.758500,0.6,0.2,0.740,0.40,0.64355
A_0,2.567,3,0.855667,0.3,0.4,0.671,0.35,0.63010
A_2,1.764,3,0.588000,0.6,0.4,0.519,0.50,0.53400


# Corat - Coret


In [126]:
import numpy as np

coba_np = np.identity(5)
a = [
     [1,2,3,4,5],
     [6,7,8,9,10],
     [11,12,13,14,15],
     [16,17,18,19,20],
     [21,22,23,24,25]
     ]
np_a = np.array(a)
coba_np

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [127]:
prob_coba = [0.1, 0.3, 0.5, 0.4, 0.5]

'''
for i in range(5):
  print(np_a[i][0])
  np_a[i][0] *= 5 

np_a
'''

'\nfor i in range(5):\n  print(np_a[i][0])\n  np_a[i][0] *= 5 \n\nnp_a\n'

In [128]:
for i in range(5):
  list_con = []
  number_con = int(np.ceil(prob_coba[i] * 5))
  for j in range(number_con):
    numb_ran = random.randint(0,4)
    if (numb_ran not in list_con and numb_ran != i):
      list_con.append(numb_ran)
    elif (numb_ran in list_con or numb_ran == i):
      while (numb_ran in list_con or numb_ran == i):
        numb_ran = random.randint(0,4)
      list_con.append(numb_ran)
  print(number_con)
  print(list_con)
  print('=============')
  for j in list_con:
    coba_np[i][j] = j+1

1
[1]
2
[0, 2]
3
[0, 3, 4]
2
[2, 1]
3
[1, 0, 3]


In [129]:
coba_np = np.identity(5)
prob_coba = [0.1, 0.3, 0.5, 0.4, 0.5]
#prob_coba = [1, 1, 1, 1, 1]
coba_np

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [130]:
for i in range(5):
  list_row = list(coba_np[i,:])

  list_pos_con = []
  list_already_con = []
  for n,val in enumerate(list_row):
    if val == 0:
      list_pos_con.append(n)
    elif val == 1 and n != i:
      list_already_con.append(n)

  #print(list_pos_con)
  number_con = int(np.ceil(prob_coba[i] * 4))

  random_con = 0
  if (len(list_already_con) < number_con):
    con_left = number_con - (4-len(list_pos_con))
    rand_con = random.sample(list_pos_con, con_left)
    rand_con = random.sample(list_pos_con, con_left)
    
    for j in rand_con:
      coba_np[j,i] = 1
      coba_np[i,j] = 1
  #if (len(list_pos_con) == 4 ):
  #  con_left = number_con
  #else:
  #  con_left = number_con - (4-len(list_pos_con))
  #print("list_post_con ")
  #print(list_pos_con)
  #print("con_left ")
  #print(con_left)
  #print("\n")
  

  

coba_np

array([[1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 1.],
       [0., 1., 1., 1., 0.],
       [1., 0., 1., 1., 0.],
       [1., 1., 0., 0., 1.]])

In [131]:
for i in range(len(coba_np)):
  for n,val in enumerate(list(coba_np[1,:])):
    if (n > i and val == 1):
      rand = round(random.uniform(0.5,1.0),3)
      coba_np[n,i] = rand
      coba_np[i,n] = rand

coba_np

array([[1.   , 0.595, 0.822, 1.   , 0.941],
       [0.595, 1.   , 0.605, 0.   , 0.664],
       [0.822, 0.605, 1.   , 1.   , 0.   ],
       [1.   , 0.   , 1.   , 1.   , 0.   ],
       [0.941, 0.664, 0.   , 0.   , 1.   ]])

In [132]:
coba_np

array([[1.   , 0.595, 0.822, 1.   , 0.941],
       [0.595, 1.   , 0.605, 0.   , 0.664],
       [0.822, 0.605, 1.   , 1.   , 0.   ],
       [1.   , 0.   , 1.   , 1.   , 0.   ],
       [0.941, 0.664, 0.   , 0.   , 1.   ]])

In [133]:
df = pd.DataFrame(columns=['node_1','node_2','quality','datetime'])
for i in range(len(coba_np)):
  list_node_rel = list(coba_np[i,:])
  for n,val in enumerate(list_node_rel):
    if (n != i and val != 0):
      df = df.append({'node_1':'A_'+ str(i), 'node_2':'A_' + str(n), 'quality':val,'datetime':dt.datetime.now()}, ignore_index = True)

In [134]:
df

,node_1,node_2,quality,datetime
0,A_0,A_1,0.595,2021-10-09 06:17:32.175895
1,A_0,A_2,0.822,2021-10-09 06:17:32.179912
2,A_0,A_3,1.000,2021-10-09 06:17:32.183560
3,A_0,A_4,0.941,2021-10-09 06:17:32.188440
4,A_1,A_0,0.595,2021-10-09 06:17:32.193312
5,A_1,A_2,0.605,2021-10-09 06:17:32.197835
6,A_1,A_4,0.664,2021-10-09 06:17:32.203051
7,A_2,A_0,0.822,2021-10-09 06:17:32.206408
8,A_2,A_1,0.605,2021-10-09 06:17:32.209903
9,A_2,A_3,1.000,2021-10-09 06:17:32.213359


In [135]:
a = 0
a = [1,2,3,4,5]
a

[1, 2, 3, 4, 5]

In [136]:
round(random.uniform(0.5,1.0),3)

0.75

In [137]:
int(np.ceil(0.0000001 * 1))

1

In [138]:
a_dict = {'color': 'blue', 'fruit': 'apple', 'pet': 'dog'}
for i in a_dict:
  print(a_dict[i])

blue
apple
dog


In [139]:
val = 3
if (val > 3):
  ls_rand_gen = [0.1, 0.15, 0.35, 0.3]
  ls_ha = []

  if (val > 4):
    for i in ls_rand_gen:
      ls_ha.append(int(np.ceil(i * val)))

    if (sum(ls_ha) > val):
      ls_ha[2] -= (sum(ls_ha) - val)
    elif (sum(ls_ha) < val):
      ls_ha[2] += (val - sum(ls_ha))

    if (ls_ha[3] > ls_ha[2]):
      ls_ha[2], ls_ha[3] = ls_ha[3], ls_ha[2]
  elif (val == 4):
    ls_ha = [1,1,1,1]

  dict_res = {}
  ls_index = range(val)
  for i,val in enumerate(ls_ha):
    res_random = random.sample(ls_index, val)
    dict_res['Class-'+str(i)]  = res_random
    ls_index = list(set(ls_index)-set(res_random))

  print(dict_res)
elif (val == 3):
  dict_res = {
      'Class-1':[0],
      'Class-2':[1],
      'Class-3':[2]
  }
  print(dict_res)
elif (val == 2):
  dict_res = {
      'Class-1':[0],
      'Class-2':[1],
  }
  print(dict_res)

{'Class-1': [0], 'Class-2': [1], 'Class-3': [2]}


In [140]:
a = [1,2,3,4,5]
b = [1,3,5]
list(set(a)-set(b))

[2, 4]

In [145]:
data = [['tom', 10], ['nick', 15], ['juli', 14]]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Name', 'Age'])
 
# print dataframe.
df

,Name,Age
0,tom,10
1,nick,15
2,juli,14


In [146]:
df.at[2, 'kelas'] = 'sd'
df['coba'] = 2*df['Age']
df

,Name,Age,kelas,coba
0,tom,10,NaN,20
1,nick,15,NaN,30
2,juli,14,sd,28
